In [ ]:
from mnist import MNIST
import time
#import csv
from multiprocessing import Pool
from functools import partial

In [ ]:
def find_average_distances(digits, test):
    distance = list()

    for i, image in enumerate(digits):
        d = 0
        for j, intensity in enumerate(image[0]):
            # d += abs(test[j] - intensity)
            d += (test[j] - intensity) * (test[j] - intensity)

        #d = math.sqrt(d)
        distance.append((d, image[1]))

    # digit = general_classify(distance)
    return distance

def find_distance(train, test):
    d = 0
    for j, intensity in enumerate(train[0]):
        d += (test[j] - intensity) * (test[j] - intensity)  # finding squared distance

    return train[1], d

def calculate_distances(train, test, pool):
    fn = partial(find_distance, test=test) # partial function will keep test img constant
    return pool.map(fn, train) #pool will go through all train imgs

def average(train):
    organized_digits = [[] for x in range(10)]
    # average_digit = [[0] * len(train[0])] * 10
    average_digit = [[0 for x in range(len(train[0][0]))] for y in range(10)]

    digit_count = [0] * 10

    for image, label in train:
        digit_count[label] += 1
        average_digit[label] = [average_digit[label][i] + image[i] for i in range(len(image))]
        organized_digits[label].append((image, label))
    final = []
    for digit, count in zip(enumerate(average_digit), digit_count):
        final.append(([pixel / count for pixel in digit[1]], digit[0]))
    # final = [[pixel / count for pixel in digit] for digit, count in zip(average_digit, digit_count)]
    return final, organized_digits


def classify(distance, k):
    neighbours = sorted(distance[:k].copy(), key=lambda x: x[1])

    for lbl, d in distance[k:]:
    # insert somewhere in the list only if the value is smaller than the largest value in the list
        if d < neighbours[-1][1]:
            for j in range(len(neighbours)):
                # if the value being looked at is bigger than the distance i insert into that index
                if neighbours[j][1] > d:
                    neighbours.insert(j, (lbl,d))
                    break
            # remove largest value from the list
            neighbours.pop(-1)

    digit = None
    digit_count = [0] * 10  # used to count the qunatity of each digit

    for i, image in neighbours:
        digit_count[i] += 1  # increments at index i, with the indexes representing each digit

    for i, j in enumerate(digit_count):  # bubble sort of digit count to sort quantities found
        top = 0
        if j > top:
            top = j
            digit = i

    return digit


def general_classify(distances_to_average_digits, r):
    labeled = sorted(distances_to_average_digits, key=lambda x: x[0])
    contenders = labeled[:r]
    digits = [j for i, j in contenders]
    return digits

In [ ]:
directory = '/home/william/Downloads/mnist'
mndata = MNIST(directory)
# load training and testing data
train_images, train_labels = mndata.load_training()
train = list()
# consolidating training labels and images into one thing, don't know why I can't just zip it but this works
for i, j in zip(train_images, train_labels):
    train.append((i, j))

test_image, test_labels = mndata.load_testing()

n = 100#len(test_image)  # number of images that classifier will test
k = 3  # number of neighbours classifier will look at
r = 3 # number of digits classifier will find distances with

results = list()
guessed_labels = list()

start_time = time.time()

# average_digits is a list of length 10, each containing the average of each digit
# grouped_digits is a list of length 10 with each index containing all images with specific digit
average_digits, grouped_digits = average(train)

results = list()

accuracy = 0
pool = Pool()
for i in range(n):
    # creates a list of the closest digits based of the average digit
    numbers = general_classify(find_average_distances(average_digits, test_image[i]), r)

    # creates list of all training digits from the 'r' digits that were closest to the test image
    filtered_train = grouped_digits[numbers[0]]
    for j in range(1, r):
        filtered_train = filtered_train + grouped_digits[numbers[j]]

    # finds the distances between all the training images selected and 
    distances = calculate_distances([x for x in filtered_train], test_image[i], pool)
    
    # classifies digit
    number = classify(distances, k)

    # used for data collection purposes
    contenders = [numbers[j] for j in range(r)]
    results.append([number, test_labels[i]] + contenders)
    print(results[i])

    if number == test_labels[i]:
        accuracy += 1

accuracy = accuracy / n
end_time = time.time() - start_time
print(f'Time: {end_time}')
print(f'Accuracy: {accuracy}')

In [ ]:
# Used for writing csv file

# with open(f'hybridResults{r}.csv', 'w', newline='') as csvfile:
#     fieldnames = ['Guessed', 'Real']
#     for i in range(r):
#         fieldnames.append(f'Contenders{i}')
#     # , 'Contender3', 'Contender4', 'Contender5'
#     writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

#     writer.writeheader()
#     for result in results:
#         out = {fieldnames[0]: result[0],
#                fieldnames[1]: result[1]}
#         for i in range(2, r+2):
#             out.update({fieldnames[i]: result[i]})

#         writer.writerow(out)

In [ ]:
guessed_results = [i[0] for i in results]
real_labels = [test_labels[i] for i in range(n)]